In [1]:
import random

# 1: with index tracking + dict

class RandomizedSet:
    def __init__(self):
        self.items = []
        self.indexes = {}

    def insert(self, n):
        if n not in self.indexes:
            self.indexes[n] = len(self.items)
            self.items.append(n)
            return True
        return False

    def remove(self, n):

        if n in self.indexes:

            # swap with last and trim items array
            last = self.items[-1]
            self.items[-1], self.items[self.indexes[n]] = \
                self.items[self.indexes[n]], self.items[-1]
            self.items.pop()
            self.indexes[last] = self.indexes[n]
            del self.indexes[n]

            return True

        return False

    def getRandom(self):
        return self.items[random.randint(0, len(self.items) - 1)]

In [2]:
s = RandomizedSet()

In [23]:
s.insert(2)
s.insert(3)

True

In [26]:
# 2: using custom hashing with chaining
class RandomizedSet:
    """RandomSet implementation using hashing with chaining

    Args:
        min_keyspace_size (int): Miniumum keyspace size below which
            it would not be shrinked even if load factor is low.
    """

    def __init__(self, min_keyspace_size=1000):
        # minimum keyspace size
        self.min_keyspace_size = min_keyspace_size

        # capacity used (effective # of items stored - including deleted)
        self.capacity = 0

        # items storage indexed by hash
        self.items = [[] for x in range(min_keyspace_size)]

        # list of hashes used
        self.used_hashes = []

        # hashing parameters
        self.P = 2038074743
        self.k = random.randint(0, (1 << 31) - 1)

    def _hash(self, item, keyspace_size):
        """Calculates integer hash for an item given keyspace size
        
        Agrs:
            item (int): item to calculate hash for
            keyspace_size (int): keyspace size
            
        Returns:
            int: hash value
        """

        return ((self.k * item) % self.P) % keyspace_size

    def _rehash(self, keyspace_size):
        """Rehashes items for new keyspace size
        
        Agrs:
            keyspace_size (int): new keyspace size
            
        Returns:
            None
        """

        # k for hashing
        self.k = random.randint(0, (1 << 31) - 1)

        self.capacity = 0
        self.used_hashes = []

        items = [[] for x in range(keyspace_size)]

        for hash_items in self.items:
            for item in hash_items:

                if not item is None:  # None means deleted items
                    h = self._hash(item, keyspace_size)

                    # keep track of occupied keys
                    if 0 == len(items[h]):
                        self.used_hashes.append(h)

                    # chain colliding items
                    items[h].append(item)
                    self.capacity += 1

        self.items = items

    def insert(self, item):
        """Insert an item
        
        Agrs:
            item (int): new item
            
        Returns:
            None
        """
        # if capacity exceeds 75%, grow keyspace by 2x
        if self.capacity > len(self.items) * 0.75:
            self._rehash(len(self.items) * 2)

        # compute hash and insert item
        h = self._hash(item, len(self.items))

        # find if item already exists
        # or can be inserted in place of deleted one
        for i in range(len(self.items[h])):
            if item == self.items[h][i]:  # already exists
                return False
            elif self.items[h][i] is None:  # reuse deleted item spot
                self.items[h][i] = item
                return

        # keep track of occupied keys
        if 0 == len(self.items[h]):
            self.used_hashes.append(h)

        self.items[h].append(item)
        self.capacity += 1

        return True

    def remove(self, item):
        """Remove an item if it exists.
        
        Agrs:
            item (int): item to remove
            
        Returns:
            None
        """

        r = False

        # compute hash
        h = self._hash(item, len(self.items))

        # go through the chain of items
        # whose hash equals h
        for i in range(len(self.items[h])):
            if item == self.items[h][i]:
                self.items[h][i] = None
                self.capacity -= 1
                r = True
                break

        # if keyspace is filled for less than 25% shrink it by half
        if len(self.items) > self.min_keyspace_size and \
            self.capacity < len(self.items) * 0.25:
            self._rehash(len(self.items) // 2)

        return r

    def getRandom(self):
        """Return random item
            
        Returns:
            int: item
        """

        item = None

        while item is None:
            # pick hash value from used hashes
            h = self.used_hashes[random.randint(0, len(self.used_hashes) - 1)]

            # pick random item from items with hash h
            item = self.items[h][random.randint(0, len(self.items[h]) - 1)]

        return item

In [35]:
s = RandomizedSet()
print(s.insert(1))
print(s.remove(2))
print(s.insert(2))
print(s.getRandom())

True
False
True
1
